In [1]:
from IPython.display import Image, display
from typing import Annotated
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from PDF_Tool import pdf_tool
from CSV_Tool import csv_tool
from prompt import prompt

In [2]:
# add memory to remeber the previos conversation
memory = MemorySaver()
# Class to handle the updated messages without overwriting


class State(TypedDict):
    messages: Annotated[list, add_messages]


# Create state graph (Workflow) and make the input is the state
graph_builder = StateGraph(State)

In [3]:
# Create tools
tools = [csv_tool, pdf_tool]
# Loading model
llm = ChatOllama(model="mistral", temperature=0)
# Nake chain between llm and tools
llm_with_tools = llm.bind_tools(tools)
tool_node = ToolNode(tools=tools)


In [4]:
def chatbot(state: State):
    # Fill the prompt template with user input
    user_message = state["messages"][-1] if state["messages"] else ""
    formatted_prompt = prompt.format(user_input=user_message[1])
    response = llm_with_tools.invoke([("user", formatted_prompt)])
    return {"messages": [response]}

In [5]:
# add chatbot to graph
graph_builder.add_node("chatbot", chatbot)
# add tool to graph
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory)

In [7]:
config = {"configurable": {"thread_id": "1"}}

user_input = "what is certifacate that bandar had, and what is degree?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()


================================ Human Message =================================

what is certifacate that bandar had, and what is degree?


TypeError: 'HumanMessage' object is not subscriptable

In [ ]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langgraph.visualization import visualize_chain

# Define your LLM
llm = OpenAI(model="gpt-4", temperature=0.7)

# Define tools (e.g., Python calculator or search API)
def basic_calculator(input: str) -> str:
    try:
        result = eval(input)
        return f"The result of {input} is {result}"
    except Exception as e:
        return str(e)

calculator_tool = Tool(
    name="Calculator",
    func=basic_calculator,
    description="A basic calculator that evaluates arithmetic expressions."
)

# Create a prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="You are an AI agent that answers questions. Here is the query: {query}"
)

# Define an LLM chain with the prompt template
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# Initialize the agent
tools = [calculator_tool]
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Build the query response
def answer_query(query: str) -> str:
    return agent.run(query)

# Visualization of the chain
visualize_chain(llm_chain)

# Example Usage
if __name__ == "__main__":
    query = "What is 2 + 2?"
    response = answer_query(query)
    print("Response:", response)
